# **Supervised Machine Learning - WALMART SALES**

## **Project**
Walmart's marketing service has asked you to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

## **Goals**

The project aims to construct a basic machine learning model for sales prediction while addressing overfitting concerns. It will be structured into three main components:

1. **Exploratory Data Analysis (EDA) of the Dataset**

2. **Data Preprocessing, Model Training, and Performance Analysis**

3. **Feature Importance and Mitigating Overfitting Issues**


In [256]:
# Library imports for project analysis
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

### 1. Exploratory Data Analysis (EDA) of the Dataset

In [257]:
# Dataset loading
print("Loading dataset...")
df = pd.read_csv("Walmart_Store_sales.csv")
print("...Done.")
print()

Loading dataset...
...Done.



In [259]:
print("Number of rows and columns:  : {}".format(df.shape))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Number of rows and columns:  : (150, 8)

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [260]:
# The goal is to predict Weekly_Sales. Missing values for this columns have been removed. 

df.dropna(subset=['Weekly_Sales'], inplace=True)
df.describe(include="all")

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,136.000000,118,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000
unique,NaN,79,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN
mean,10.014706,NaN,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582
std,6.124614,NaN,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000
50%,10.000000,NaN,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000
75%,15.250000,NaN,1.806386e+06,0.000000,75.950000,3.724000,214.878556,8.150000


In [261]:
# Outliers removal / + or - 3 std. 

print('Dropping outliers in Temperature, Fuel_Price, CPI and Unemployment')

list1 = ['Temperature', 'Fuel_Price', 'CPI' , 'Unemployment']

for el in list1 : 
    to_keep = (df[el] < df[el].mean() + 3*df[el].std()) & (df[el] > df[el].mean() - 3*df[el].std())
    df = df.loc[to_keep,:]

print('Done. Number of lines remaining : ', df.shape[0])
print()

df.describe()

Dropping outliers in Temperature, Fuel_Price, CPI and Unemployment
Done. Number of lines remaining :  90



,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,90.000000,9.000000e+01,80.000000,90.00000,90.000000,90.000000,90.000000
mean,9.900000,1.233865e+06,0.075000,61.06100,3.318444,179.524905,7.389733
std,6.204475,6.647250e+05,0.265053,17.74604,0.484399,39.554303,0.982729
min,1.000000,2.689290e+05,0.000000,18.79000,2.548000,126.128355,5.143000
25%,4.000000,5.617240e+05,0.000000,45.34250,2.814750,132.602339,6.642250
50%,9.000000,1.260826e+06,0.000000,61.45000,3.468000,197.166416,7.419000
75%,15.750000,1.807159e+06,0.000000,75.79250,3.737750,214.855374,8.099000
max,20.000000,2.771397e+06,1.000000,91.65000,4.170000,226.968844,9.342000


In [262]:
# Date management

print(type(df["Date"][1]))

df["Date"] = df["Date"].apply(pd.to_datetime)

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

df = df.drop(columns=["Date"])

# Removal of missing values
df.dropna(subset=['Year'], inplace=True)

df.describe()

<class 'str'>


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day
count,80.000000,8.000000e+01,71.000000,80.00000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000
mean,9.575000,1.221522e+06,0.084507,61.12775,3.290700,181.077638,7.301775,2010.887500,6.525000,15.962500
std,6.143382,6.799270e+05,0.280126,17.44760,0.491223,38.847021,0.955392,0.826672,3.329861,8.594145
min,1.000000,2.689290e+05,0.000000,18.79000,2.548000,126.139200,5.143000,2010.000000,1.000000,1.000000
25%,4.000000,5.295107e+05,0.000000,45.58750,2.804000,132.610242,6.520750,2010.000000,4.000000,8.000000
50%,8.000000,1.260826e+06,0.000000,61.45000,3.390500,197.500965,7.345500,2011.000000,6.000000,16.500000
75%,15.000000,1.817517e+06,0.000000,75.47750,3.689750,214.809008,8.090000,2012.000000,9.250000,23.250000
max,20.000000,2.771397e+06,1.000000,91.65000,4.170000,226.968844,9.342000,2012.000000,12.000000,31.000000


In [263]:
# Let's run a bivariate analysis

fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False,
            autosize=False, height=1000, width = 1000)
fig.show()

In [264]:
# Let's add correlation matrix
corr_matrix = df.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

* Regarding this matrix, few features are totally correlated. 
* Year and Fuel_Price are the most correlated features.

In [265]:
# Creating scatter plots to visualize the relationship between Weekly_Sales and some features of the dataset. 

fig1 = px.scatter(df, x="CPI", y="Weekly_Sales")
fig2 = px.scatter(df, x="Fuel_Price", y="Weekly_Sales")
fig3 = px.scatter(df, x="Unemployment", y="Weekly_Sales")

fig1.show()
fig2.show()
fig3.show()

### 2. Data Preprocessing, Model Training, and Performance Analysis

a) Preprocessing

In [266]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Weekly_Sales"


X = df.drop(target_variable, axis = 1)
Y = df.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

X :
   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
0    6.0           NaN        59.61       3.045  214.777523         6.858   
1   13.0           0.0        42.38       3.435  128.616064         7.470   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
6   15.0           0.0        69.80       4.069  134.855161         7.658   
7   20.0           0.0        39.93       3.617  213.023622         6.961   

     Year  Month   Day  
0  2011.0    2.0  18.0  
1  2011.0    3.0  25.0  
4  2010.0    5.0  28.0  
6  2011.0    3.0   6.0  
7  2012.0    3.0   2.0  


In [267]:
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [268]:
# Identification of numeric and categorical features

numeric_features = []
categorical_features = []
for i,t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day']
Found categorical features  []


All variables are numeric. However, 'Store' and 'Holiday_Flag' are discrete, so we will treat them as categorical variables.

In [269]:
# Standardize numeric features by removing the mean and scaling to unit variance.
numeric_features = ["Temperature","Fuel_Price", "CPI", "Unemployment", "Day", "Month", "Year"]  
numeric_transformer = StandardScaler()

# Some value in categorical features are missing. We replace them by the most frequent value. Then we use OneHotEncoder to create a binary column for each category.
categorical_features = ['Store', 'Holiday_Flag']
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('encoder', OneHotEncoder(drop='first')) 
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [270]:
# Application of all preprocessing 

print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) 


print('...Done.')
print(X_test[0:5,:]) 
print()

Performing preprocessings on train set...
    Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
6    15.0           0.0        69.80       4.069  134.855161         7.658   
38    4.0           0.0        81.85       3.570  129.066300         5.946   
12    7.0           0.0        36.61       3.767  192.826069         8.595   
44    1.0           1.0        38.51       2.548  211.242170         8.106   
66   18.0           0.0        73.67       2.792  132.614193         9.342   

      Year  Month   Day  
6   2011.0    3.0   6.0  
38  2011.0    6.0  24.0  
12  2011.0    5.0  13.0  
44  2010.0   12.0   2.0  
66  2010.0    6.0   8.0  
...Done.
[[ 0.58589224  1.56199819 -1.05536176  0.30814094 -1.10218284 -1.00659072
   0.18751465  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          1.          0.          0.          0.          0.
   0.          0.        ]
 [ 1.28430802  0.5704465 

b. Model Training

In [271]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


c. Performance analysis

In [272]:
print(f"Train score (R2): {regressor.score(X_train, Y_train)}")
print(f"Test score (R2): {regressor.score(X_test, Y_test)}")

Train score (R2): 0.9822844283170552
Test score (R2): 0.9642563359944234


* These scores seem to be very good (R2 is close to 1). However, the score on the training set is higher than the score on the test set. This difference may be indicative of overfitting.
* We will perform cross-validation to assess the standard deviation arising from the splitting of the training and test sets.

In [273]:
print("3-fold cross-validation...")
scores = cross_val_score(regressor, X_train, Y_train, cv=10)
print('The cross-validated accuracy-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated accuracy-score is :  0.9372034927668287
The standard deviation is :  0.03227668399207734


**Conclusion** 
* In our case, we can conclude that the overfitting of our model is not blatant since if we subtract the standard deviation from the training score, we fall within the range of the test score.
* However, we will still apply regularization to see what result we obtain.

### 3. Features Importance and Mitigating Overfitting Issues

a) Features Importance

In [274]:
# Focus on coefficients of the model
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names

print("Names of columns corresponding to each coefficient: ", column_names)



coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])

# Sort value to obtain the weight of each features 
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Day', 'Month', 'Year', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x0_10.0', 'x0_11.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_20.0', 'x1_1.0']


,coefficients
Month,2.347013e+04
Temperature,4.527777e+04
Day,4.641716e+04
x0_6.0,5.448072e+04
Fuel_Price,7.632217e+04
x0_16.0,9.398510e+04
Unemployment,9.686075e+04
x0_7.0,1.142121e+05
x0_11.0,1.352437e+05
Year,1.382563e+05


In [275]:
# Let's display a graph
fig = px.bar(coefs, barmode="group")

fig.show()

* The analysis of model coefficients highlights the relative impact of various features on weekly sales. 
* Notably, store numbers (specifically, stores 4.0, 10.0, and 13.0) demonstrate significant influence, followed by factors like the year, unemployment rate, and consumer price index (CPI). 
* However, certain factors, like temperature or fuel price, appear to have limited influence on the model's predictions.

b) Mititing Overfitting Issues

In [276]:
# Regularization with Ridge
regressor = Ridge()
params = {
    'alpha': [0.1, 0.5, 1, 2, 3, 5, 10, 20, 30, 100, 1000] # 0 corresponds to no regularization
}
gridsearch_ridge = GridSearchCV(regressor, param_grid = params, cv = 5, verbose = 1) 
gridsearch_ridge.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_ridge.best_params_)
print("Best R2 score : ", gridsearch_ridge.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.8962691468370017


In [277]:
# Regularization with Lasso
regressor = Lasso(max_iter=100000)
params = {
    'alpha': [0.1, 0.5, 1, 2, 3, 5, 10, 20, 30, 100, 1000] # 0 corresponds to no regularization
}
gridsearch_lasso = GridSearchCV(regressor, param_grid = params, cv = 5, verbose=1) # cv : the number of folds to be used for CV
gridsearch_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R2 score : ", gridsearch_lasso.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
...Done.
Best hyperparameters :  {'alpha': 0.1}
Best R2 score :  0.9396934318568174


In [278]:
print("RIDGE / R2 score on training set : ", gridsearch_ridge.score(X_train, Y_train))
print("RIDGE / R2 score on test set : ", gridsearch_ridge.score(X_test, Y_test))
print()
print("LASSO / R2 score on training set : ", gridsearch_lasso.score(X_train, Y_train))
print("LASSO / R2 score on test set : ", gridsearch_lasso.score(X_test, Y_test))

RIDGE / R2 score on training set :  0.9753202538068639
RIDGE / R2 score on test set :  0.9752320372531884

LASSO / R2 score on training set :  0.9822844134224604
LASSO / R2 score on test set :  0.964321264853796


### **Conclusion**

* Thanks to both regularization techniques (Ridge and Lasso), we achieve test and train scores that are much closer, with notably better performance from the Ridge regularization. The Ridge model demonstrates an R2 score of 0.9752 on the test set, indicating good generalization to new data. On the other hand, while the Lasso model shows excellent fit to the training data with an R2 score of 0.9823, its performance on the test set is slightly lower (R2 score of 0.9643). This difference can be attributed to the sometimes aggressive nature of variable selection by Lasso, which may lead to less effective generalization to new data. 
* Thus, for deployment in production, we favor the Ridge model, deemed more robust against overfitting and providing better overall performance.
